## Setup & Imports

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from typing import Dict, List, Any
from datetime import datetime
import json

# Import from the package
try:
    from llm_auction_optimization import (
        ClinicalVignette,
        PrivateAssessment,
        AuctionResult,
        Intervention,
        RoundOutcome,
        AgentState,
        run_auction,
        ExperimentConfig,
        ExperimentLogger,
        ExperimentMetrics,
        MetricsDisplay,
    )
except ImportError as e:
    print(f"Note: Could not import from package. Will use inline implementations.")
    print(f"Error: {e}")

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Imports successful")

## Part 1: Create Mock Clinical Vignettes

For MVP, we'll use synthetic vignettes. In production, these come from the Ethical Reasoning in Mental Health HuggingFace dataset.

In [ ]:
# Mock clinical vignettes for demo
mock_vignettes = [
    {
        'id': 'V1',
        'text': '32-year-old male presents with fever (39°C), cough, and dyspnea for 3 days. CXR shows left lower lobe infiltrate. No significant PMHx.',
        'options': ['Pneumonia', 'Bronchitis', 'Asthma exacerbation', 'Pulmonary embolism'],
        'ground_truth': 'Pneumonia',
        'triage': 'ER',
    },
    {
        'id': 'V2',
        'text': '58-year-old female with chest pain, dyspnea, and diaphoresis. EKG shows ST elevation in II, III, aVF.',
        'options': ['Acute MI', 'Unstable angina', 'Aortic dissection', 'Pneumonia'],
        'ground_truth': 'Acute MI',
        'triage': 'ER',
    },
    {
        'id': 'V3',
        'text': '24-year-old female with mild headache and no fever. Exam unremarkable. Works in busy office.',
        'options': ['Migraine', 'Tension headache', 'Meningitis', 'Subarachnoid hemorrhage'],
        'ground_truth': 'Tension headache',
        'triage': 'Routine',
    },
    {
        'id': 'V4',
        'text': '45-year-old male with acute onset severe abdominal pain, vomiting. Fever 38.5°C. Rebound tenderness.',
        'options': ['Appendicitis', 'Gastroenteritis', 'Pancreatitis', 'Diverticulitis'],
        'ground_truth': 'Appendicitis',
        'triage': 'ER',
    },
    {
        'id': 'V5',
        'text': '72-year-old female with chronic leg swelling. No acute pain. Ambulatory. Mild weight gain.',
        'options': ['DVT', 'Cellulitis', 'Venous insufficiency', 'Lymphedema'],
        'ground_truth': 'Venous insufficiency',
        'triage': 'Routine',
    },
]

# Convert to ClinicalVignette objects
vignettes = [
    ClinicalVignette(
        vignette_id=v['id'],
        text=v['text'],
        ground_truth_action=v['ground_truth'],
        options=v['options'],
        task_type='diagnosis',
        metadata={'triage': v['triage']}
    )
    for v in mock_vignettes
]

print(f"Created {len(vignettes)} clinical vignettes")
print("\nExample vignette:")
print(f"  ID: {vignettes[0].vignette_id}")
print(f"  Text: {vignettes[0].text}")
print(f"  Ground truth: {vignettes[0].ground_truth_action}")
print(f"  Options: {vignettes[0].options}")

## Part 2: Define Mock Agents

In production, these would call actual LLMs. For MVP, we use mock agents with synthetic behavior patterns.

In [ ]:
class MockClinicalAgent:
    """Mock agent that simulates LLM behavior for clinical decision-making."""
    
    def __init__(self, agent_id: str, style: str = 'neutral'):
        self.agent_id = agent_id
        self.style = style
        self.budget = 1.0
        self.state = AgentState(
            agent_id=agent_id,
            initial_budget=1.0,
            remaining_budget=1.0,
            communication_style=style
        )
    
    def assess(self, vignette: ClinicalVignette) -> PrivateAssessment:
        """Independently assess the vignette."""
        # Mock: deterministic but with noise based on style
        np.random.seed(hash(self.agent_id + vignette.vignette_id) % 2**32)
        
        # Correct answer with some probability
        if np.random.random() < 0.6:  # 60% accuracy baseline
            action = vignette.ground_truth_action
        else:
            action = np.random.choice([a for a in vignette.options if a != vignette.ground_truth_action])
        
        # Confidence based on style
        if self.style == 'overconfident':
            confidence = min(0.95, np.random.normal(0.8, 0.1))
        elif self.style == 'underconfident':
            confidence = max(0.3, np.random.normal(0.5, 0.1))
        else:
            confidence = np.clip(np.random.normal(0.65, 0.15), 0.3, 0.95)
        
        # Rationale (mock)
        rationale = f"• Key finding: {vignette.text.split()[0:5]}\n• Differential: {action} or alternative\n• Recommendation: {action}"
        
        return PrivateAssessment(
            agent_id=self.agent_id,
            recommended_action=action,
            confidence=float(confidence),
            rationale=rationale
        )
    
    def bid(self, assessment: PrivateAssessment) -> float:
        """Decide how much to bid for proposal rights."""
        # Bid based on confidence and style
        base_bid = min(self.budget, assessment.confidence * 0.5)
        
        if self.style == 'assertive':
            bid = base_bid * 1.3
        elif self.style == 'timid':
            bid = base_bid * 0.7
        else:
            bid = base_bid
        
        return min(self.budget, bid)  # Cap at budget
    
    def should_speak(self, proposer_action: str, vignette: ClinicalVignette, p=0.4) -> bool:
        """Decide whether to pay to speak."""
        # Speak if proposer might be wrong
        if self.style == 'cooperative':
            return np.random.random() < (p * 1.3)
        elif self.style == 'competitive':
            return np.random.random() < (p * 0.7)
        else:
            return np.random.random() < p
    
    def vote(self, all_options: Dict[str, str], vignette: ClinicalVignette) -> str:
        """Vote on best option."""
        # Vote for own if available, else random
        if self.agent_id in all_options:
            if self.style == 'competitive':
                return all_options[self.agent_id]  # Always vote for self
        # Random vote among options
        return np.random.choice(list(all_options.values()))

print("✓ MockClinicalAgent defined")

## Part 3: Implement Core Auction Game Loop

In [ ]:
def run_clinical_auction_round(
    agents: List[MockClinicalAgent],
    vignette: ClinicalVignette,
    token_price: float = 0.001,
    reward_amount: float = 1.0,
    auction_type: str = 'sealed_bid',
    round_id: int = 0
) -> RoundOutcome:
    """
    Run one complete auction round for clinical decision-making.
    """
    
    # ROUND 0: Private Assessment
    private_assessments = {}
    for agent in agents:
        assessment = agent.assess(vignette)
        private_assessments[agent.agent_id] = assessment
    
    # ROUND 1: Sealed-Bid Auction for Proposer Rights
    bids = {}
    for agent in agents:
        bid = agent.bid(private_assessments[agent.agent_id])
        bids[agent.agent_id] = min(bid, agent.state.remaining_budget)
    
    # Run auction
    proposer_id, proposer_cost = run_auction(bids, auction_type, {a.agent_id: a.state.remaining_budget for a in agents})
    
    # Deduct bid cost
    for agent in agents:
        if agent.agent_id == proposer_id:
            agent.state.remaining_budget -= proposer_cost
            agent.state.total_bids_paid += proposer_cost
    
    proposer_assessment = private_assessments[proposer_id]
    
    # ROUND 2: Optional Paid Interventions
    interventions = []
    all_options = {proposer_id: proposer_assessment.recommended_action}
    
    for agent in agents:
        if agent.agent_id != proposer_id:
            if agent.should_speak(proposer_assessment.recommended_action, vignette):
                # Generate a mock message
                message = f"I suggest {np.random.choice(vignette.options)}"
                token_count = len(message.split())
                speaking_cost = token_count * token_price
                
                if speaking_cost <= agent.state.remaining_budget:
                    agent.state.remaining_budget -= speaking_cost
                    agent.state.total_tokens_used += token_count
                    agent.state.interventions_made += 1
                    
                    # Extract alternative action from message
                    alt_action = message.split()[-1]  # Last word
                    if alt_action in vignette.options:
                        all_options[agent.agent_id] = alt_action
                    
                    interventions.append(Intervention(
                        agent_id=agent.agent_id,
                        message=message,
                        token_count=token_count,
                        cost=speaking_cost,
                        suggested_alternative_action=alt_action if alt_action in vignette.options else None
                    ))
    
    # ROUND 3: Vote
    votes = {}
    for agent in agents:
        vote = agent.vote(all_options, vignette)
        votes[agent.agent_id] = vote
    
    # Majority vote
    vote_counts = Counter(votes.values())
    final_action = vote_counts.most_common(1)[0][0]
    
    # ROUND 4: Evaluate & Payoff
    correctness = (final_action == vignette.ground_truth_action)
    
    agent_rewards = {}
    for agent in agents:
        agent.state.rounds_participated += 1
        
        if agent.agent_id == proposer_id:
            agent.state.times_proposer += 1
            if proposer_assessment.recommended_action == vignette.ground_truth_action:
                agent.state.times_proposer_correct += 1
        
        if correctness:
            reward = reward_amount
            
            # Bonus for helpful interventions
            for intervention in interventions:
                if intervention.agent_id == agent.agent_id and intervention.suggested_alternative_action == vignette.ground_truth_action:
                    reward += 0.25
                    agent.state.interventions_valuable += 1
        else:
            reward = 0.0
        
        agent.state.cumulative_reward += reward
        agent_rewards[agent.agent_id] = reward
    
    # Compute totals
    total_tokens = sum(i.token_count for i in interventions)
    total_cost = proposer_cost + sum(i.cost for i in interventions)
    
    # Create outcome
    outcome = RoundOutcome(
        timestamp=datetime.now(),
        vignette=vignette,
        round_id=round_id,
        private_assessments=private_assessments,
        auction_result=AuctionResult(
            proposer_id=proposer_id,
            proposer_bid=bids[proposer_id],
            all_bids=bids
        ),
        interventions=interventions,
        votes=votes,
        final_action=final_action,
        ground_truth_action=vignette.ground_truth_action,
        correctness=correctness,
        total_tokens=total_tokens,
        total_cost=total_cost,
        agent_rewards=agent_rewards
    )
    
    return outcome

print("✓ Game loop implemented")

## Part 4: Run MVP Demo

Let's run a few vignettes with the auction mechanism and compare against a baseline (free discussion).

In [ ]:
# Create agents with different styles
styles = ['assertive', 'timid', 'overconfident', 'cooperative', 'neutral']
agents = [MockClinicalAgent(f'Agent_{i+1}', style=styles[i]) for i in range(5)]

print("Agents created:")
for agent in agents:
    print(f"  {agent.agent_id}: {agent.style}")

# Run auction condition on first 3 vignettes
print("\n" + "="*60)
print("Running AUCTION condition (budgeted communication)")
print("="*60)

auction_outcomes = []
for i, vignette in enumerate(vignettes[:3]):
    # Reset agents for next round
    for agent in agents:
        agent.state.remaining_budget = 1.0
    
    outcome = run_clinical_auction_round(
        agents=agents,
        vignette=vignette,
        token_price=0.001,
        reward_amount=1.0,
        auction_type='sealed_bid',
        round_id=i
    )
    auction_outcomes.append(outcome)
    
    print(f"\nVignette {i+1}: {vignette.vignette_id}")
    print(f"  Ground truth: {vignette.ground_truth_action}")
    print(f"  Proposer: {outcome.auction_result.proposer_id} (bid: ${outcome.auction_result.proposer_bid:.4f})")
    print(f"  Proposer action: {outcome.private_assessments[outcome.auction_result.proposer_id].recommended_action}")
    print(f"  Interventions: {len(outcome.interventions)}")
    print(f"  Final action: {outcome.final_action}")
    print(f"  Correct: {outcome.correctness}")
    print(f"  Total cost: ${outcome.total_cost:.4f} (tokens: {outcome.total_tokens})")

print("\n✓ Auction demo complete")

## Part 5: Results & Analysis

In [ ]:
# Compute metrics for auction condition
metrics_auction = ExperimentMetrics.compute_metrics(auction_outcomes)
agent_stats = ExperimentMetrics.compute_agent_stats({a.agent_id: a.state for a in agents})

print("\nAuction Condition Metrics:")
for key, value in metrics_auction.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.3f}")
    else:
        print(f"  {key}: {value}")

print("\nPer-Agent Stats:")
for agent_id, stats in agent_stats.items():
    print(f"\n  {agent_id}:")
    print(f"    Rounds: {stats['rounds_participated']}")
    print(f"    Times proposer: {stats['times_proposer']}")
    print(f"    Interventions: {stats['interventions_made']}")
    print(f"    Net reward: ${stats['cumulative_reward']:.2f}")
    print(f"    Tokens used: {stats['total_tokens_used']}")

## Part 6: Visualization

In [ ]:
# Summary visualization
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 1. Correctness over rounds
ax = axes[0, 0]
correctness = [o.correctness for o in auction_outcomes]
ax.plot(range(len(correctness)), correctness, marker='o', linewidth=2, markersize=8)
ax.set_xlabel('Vignette')
ax.set_ylabel('Correct')
ax.set_title('Correctness Over Rounds (Auction)')
ax.set_ylim(-0.1, 1.1)
ax.grid(alpha=0.3)

# 2. Cost per round
ax = axes[0, 1]
costs = [o.total_cost for o in auction_outcomes]
ax.bar(range(len(costs)), costs, color='steelblue', alpha=0.7)
ax.set_xlabel('Vignette')
ax.set_ylabel('Total Cost ($)')
ax.set_title('Communication Cost per Round')
ax.grid(alpha=0.3, axis='y')

# 3. Agent participation
ax = axes[1, 0]
agent_ids = [a.agent_id for a in agents]
interventions = [agent_stats[a]['interventions_made'] for a in agent_ids]
ax.bar(agent_ids, interventions, color='coral', alpha=0.7)
ax.set_xlabel('Agent')
ax.set_ylabel('Interventions Made')
ax.set_title('Agent Participation (Paid Interventions)')
ax.grid(alpha=0.3, axis='y')

# 4. Agent rewards
ax = axes[1, 1]
rewards = [agent_stats[a]['cumulative_reward'] for a in agent_ids]
colors = ['green' if r > 0 else 'red' for r in rewards]
ax.bar(agent_ids, rewards, color=colors, alpha=0.7)
ax.set_xlabel('Agent')
ax.set_ylabel('Net Reward ($)')
ax.set_title('Cumulative Agent Rewards')
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('auction_demo.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Visualization saved to auction_demo.png")

## Next Steps for Full Implementation

This MVP demonstrates the core game mechanics. To scale up:

1. **Load Real Dataset**: Use Ethical Reasoning in Mental Health from HuggingFace
2. **Integrate Real LLMs**: Replace mock agents with OpenAI API or local models (Qwen, Llama)
3. **Run Baselines**: Implement free-discussion and turn-taking conditions
4. **Token Price Sweep**: Run experiments across multiple token prices (0.0001 to 0.01)
5. **Generate Results Plots**: Accuracy vs Cost across conditions
6. **Analyze Safety**: Count safety interventions that prevented harmful decisions

See `DESIGN.md` for detailed implementation guidance.